In this tutorial we will be looking at a database called Chinook which contains information about a fictional music shop. The database includes several tables on invoice, track, album, artist etc. related to the store's sales. We will use this database to explore four fictional business questions and propositions.

# Preparation

Firstly we'll need to import `asqlcell` and load extension properly.

In [ ]:
import asqlcell

%load_ext asqlcell

Now we can create a connection object to the DuckDB database file. Notice that the connection object will be used for SQL queries later.

In [ ]:
from sqlalchemy import create_engine, inspect

con = create_engine(f"duckdb:///chinook.duckdb").connect()

We can take a look at the database's schema via the connection object to understand which tables it contains:

In [ ]:
inspect(con).get_table_names()

The Entity Relationship Diagram of the database is provided as follows:

![Chinook Database](https://m-soro.github.io/Business-Analytics/SQL-for-Data-Analysis/L4-Project-Query-Music-Store/Misc/001.png)

# Questions

## Which countries have the most invoices?

We could use the `Invoice` table to determine the countries that have the most invoices by providing a table of `BillingCountry` and `Invoices` ordered by the number of invoices for each country. ALso, the country with the most invoices should appear first.

In [ ]:
%%sql --con con

SELECT
    BillingCountry,
    COUNT(InvoiceId) AS Invoices
FROM Invoice
GROUP BY 1
ORDER BY 2 DESC

## Which city has the best customers?

We would like to throw a promotional Music Festival in the city we made the most money.

We could write a query that returns the first city that has the highest sum of invoice totals. Return both the city name and the sum of all invoice totals.

In [ ]:
%%sql --con con

SELECT
    BillingCity,
    SUM(Total) AS Total
FROM Invoice
GROUP BY 1
ORDER BY 2 DESC

## Who is the best customer?

The customer who has spent the most money will be declared the best customer.

We could build a query that returns the person who has spent the most money. The solution is to link `Invoice` and `Customer` tables to retrieve this information.

In [ ]:
%%sql --con con

SELECT
    C.CustomerId,
    C.FirstName || ' ' || C.LastName AS Customer,
    SUM(I.Total) AS Total
FROM Customer C
JOIN Invoice I ON C.CustomerId = I.CustomerId
GROUP BY 1, 2
ORDER BY 3 DESC

## Who is writing the rock music?

Now that we know that our customers love rock music, we can decide which musicians to invite to play at the concert.

Let’s invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands. We will need to use the Genre, Track , Album, and Artist tables.

In [ ]:
%%sql --con con

SELECT
    AR.Name,
    COUNT(T.Name) AS Count
FROM Track T
JOIN Genre G ON T.GenreId = G.GenreId
JOIN Album AL ON AL.AlbumId = T.AlbumId
JOIN Artist AR ON AR.ArtistId = AL.ArtistId
WHERE G.Name = 'Rock'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10